# Estadística descriptiva

El análisis exploratorio nos ayuda a entender los datos a través de estadísticos y resúmenes de sus características,generalmente antes de la parte visual. Generalmente, suele ser acompañado de gráficos, aunque en algunos casos, esto suele causar demoras en el análisis. En la estadística descriptiva se conocen tres faces:
* Análisis univariado
* Análisis bivariado
* Análisis multivariado (más de tres características)

## Carga de archivos en python

En python existen algunas dependencias que permiten la carga de archivos en diferentes formatos (csv, txt, xlsx, json, parquet, etc.), algunas de estas se especializan en la carga de grandes volúmenes de datos, generalmente, cuando se abordan los temas teóricos y prácticos de la ciencia de datos, los datasets que se utilizan no suelen ser muy masivos. Por lo que usaremos la famosa librería `pandas` para el tratamiento de la información. Estas librerías (aprovechando la comunidad de python, suelen tener muy buena documentación, por ejemplo, eche un vistazo a la documentación de [`pandas`](https://pandas.pydata.org/docs/getting_started/intro_tutorials/index.html). A continuación, la importamos:

In [8]:
import pandas as pd
import numpy as np

Usaremos para introducir los conceptos de estadística descriptiva, el dataset de _cars_, que contiene información de características de 10 mil vehículos y se puede encontrar en [_kaggle_](https://www.kaggle.com/datasets/CooperUnion/cardataset?resource=download). Y cuya metadata adjuntamos:

**Context**

Cars dataset with features including make, model, year, engine, and other properties of the car used to predict its price.

**Content**

Scraped from Edmunds and Twitter.

**Acknowledgements**

Edmunds and Twitter for providing info
Sam Keene

**Inspiration**

Effects of features on the price
How does the brand affect the price?
What cars can be consider overpriced?
Price VS. popularity?

In [3]:
cars = pd.read_csv('cars.csv')

Ahora exploraremos la base de datos cargada.

In [4]:
cars.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [5]:
cars.dtypes

Make                  object
Model                 object
Year                   int64
Engine Fuel Type      object
Engine HP            float64
Engine Cylinders     float64
Transmission Type     object
Driven_Wheels         object
Number of Doors      float64
Market Category       object
Vehicle Size          object
Vehicle Style         object
highway MPG            int64
city mpg               int64
Popularity             int64
MSRP                   int64
dtype: object

## Análisis univariado

Para variables cuantitativas:
- Medidas de tendencia central
- Medidas de posición
- Medidas de dispersión
- Medidas de forma

### Medidas de tendencia central

Entre las medidas de tendencia central se encuentra la media, mediana y moda, y algunas variantes de las mismas: media recortada, media ponderada. Calculemos estas funciones para los datos numéricos.

In [33]:
def get_numeric(data):
    return [x for x in data.columns if data.dtypes[x] == np.float64]

In [18]:
from collections import Counter
def my_mode(sample):
    c = Counter(sample)
    return [k for k, v in c.items() if v == c.most_common(1)[0][1]][0]

In [21]:
def compute_mtc(data):
    numeric_cols = get_numeric(data)
    data_mtc = data.copy()[numeric_cols].aggregate({"mean", "median", my_mode})
    return data_mtc

In [22]:
compute_mtc(cars)

,Engine HP,Engine Cylinders,Number of Doors
my_mode,200.00000,4.000000,4.000000
mean,249.38607,5.628829,3.436093
median,227.00000,6.000000,4.000000


Las medidas de tendencia central nos brindan información sobre cómo se distribuyen los datos, los posibles sesgos que tienen.

### Medidas de dispersión

Las medidas de dispersión nos hablan de qué tan dispersos son las distribuciones, si es confiable representarlas con un parámetro o no.

In [70]:
def range_of_list(sample):
    rango = np.max(sample.dropna().values) - np.min(sample.dropna().values)
    return rango

def iqr(sample):
    q3, q1 = np.percentile(sample.dropna().values, [75 ,25])
    inqr = q3 - q1
    return inqr

def cve(sample):
    cve =  np.round(sample.std()/sample.mean(),3)
    return cve*100


In [71]:
def compute_mdd(data):
    numeric_cols = get_numeric(data)
    data_mdd = data.copy()[numeric_cols].aggregate({"std", "var", range_of_list, iqr, cve})
    return data_mdd

In [72]:
compute_mdd(cars)

,Engine HP,Engine Cylinders,Number of Doors
var,11922.864531,3.170392,0.776717
iqr,130.000000,2.000000,2.000000
std,109.191870,1.780559,0.881315
cve,43.800000,31.600000,25.600000
range_of_list,946.000000,16.000000,2.000000
